In [1]:
import json
from pprint import pprint
import copy
import re

In [8]:
with open('../../minecraft/python/craftassist/text_to_tree_tool/turk_data/final_prompts_and_humanbot/all_prompts_final.json') as f:
    data =  json.load(f)

In [9]:
print(len(data))
print(data[50])

4532
[['build a wall around the house'], {'dialogue_type': 'HUMAN_GIVE_COMMAND', 'action_sequence': [{'action_type': 'BUILD', 'location': {'location_type': 'REFERENCE_OBJECT', 'relative_direction': 'AROUND', 'reference_object': {'has_name': [0, [5, 5]]}}, 'schematic': {'has_name': [0, [2, 2]]}}]}]


In [13]:
with open('../old_data/all_data_newdct.json') as f:
    data = json.load(f)


In [24]:
import spacy
from spacy.lang.en import English
tokenizer = English().Defaults.create_tokenizer()

def tokenize(st):
    return ' '.join([str(x) for x in tokenizer(st)])

In [28]:
print(data.keys())
# do also for templated
train_data, valid_data, test_data = data['train'], data['valid'], data['test']
print(train_data.keys())
print(valid_data.keys())
print(test_data.keys())
total = {}
for data_type in [train_data, valid_data, test_data]:
    for key in ['templated', 'rephrased', 'prompts', 'humanbot']:
        if key in data_type:
            total[key] = total.get(key, 0) + len(data_type[key])
print(total)

[['kindly proceed forty six paces to the left of the yard'], {'dialogue_type': 'HUMAN_GIVE_COMMAND', 'action_sequence': [{'location': {'reference_object': {'has_name': [0, [10, 10]]}, 'steps': [0, [2, 3]], 'relative_direction': 'LEFT', 'location_type': 'REFERENCE_OBJECT'}, 'action_type': 'MOVE'}]}]


In [16]:
def to_snake_case(word, case="lower"):
    """convert a given word to snake case"""
    s1 = re.sub("(.)([A-Z][a-z]+)", r"\1_\2", word)
    snake_case = re.sub("([a-z0-9])([A-Z])", r"\1_\2", s1)
    if case == "upper":
        return snake_case.upper()
    return snake_case.lower()

In [17]:
def fix_keys(d):
    for key, val in d.items():
        if key in ['action_reference_object', 'location_reference_object']:
            d['reference_object'] = d[key]
            del d[key]
        elif key.endswith('_'):
            d[key[:-1]] = d[key]
            del d[key]
        if type(val) == dict:
            fix_keys(val)
        
    return d

In [18]:
def fix_vals(d):
    for key, val in d.items():
        if type(val) == dict:
            fix_vals(val)
        # fix spans
        elif type(val) == list:
            d[key] = [0, val]
        else:
            d[key] = to_snake_case(val, "upper")
    return d

In [19]:
def check_loc_type(d):
    for key, val in d.items():
        if key == "location_type" and val in ['SpeakerLook', 'AgentPos', 'SpeakerPos']:
            return True
        if type(val) == dict:
            if check_loc_type(val):
                return True
    return False

In [20]:
COLOURS = ["red", "blue", "green", "yellow", "purple", "grey", "brown", "black", "orange"]
ABSTRACT_SIZE = [
    "really tiny",
    "very tiny",
    "tiny",
    "really small",
    "very small",
    "really little",
    "very little",
    "small",
    "little",
    "medium",
    "medium sized",
    "big",
    "large",
    "really big",
    "very big",
    "really large",
    "very large",
    "huge",
    "gigantic",
    "really huge",
    "very huge",
    "really gigantic",
    "very gigantic",
]


In [21]:
def make_new_data(datalist):
    new_data = []

    skip_data = []

    for data_point in datalist:
        text = data_point[0]
        text = tokenize(text)
        re_dict = data_point[1]
        
        # skip the ones that need re-annotation with coref-type
        if check_loc_type(re_dict):
            skip_data += [([text], re_dict)]
        else:
            x = copy.deepcopy(re_dict)
            new_dict = fix_keys(x)
            n_dict = fix_vals(new_dict)
            updated_dict = {}

            if n_dict['action_type'] == 'TAG':
                updated_dict['dialogue_type'] = 'PUT_MEMORY'
                if 'reference_object' in n_dict:
                    updated_dict['filters'] = {'reference_object' : n_dict['reference_object']}
                if 'tag' in n_dict:
                    updated_dict['upsert'] = {'memory_data' : {"memory_type": 'TRIPLE'}}
                    tag_range = n_dict['tag'][1]
                    words = " ".join(text.split()[tag_range[0]: tag_range[1]+1])
                    if words.strip() in COLOURS:
                        updated_dict['upsert']['memory_data']['has_colour'] = n_dict['tag']
                    elif words.strip() in ABSTRACT_SIZE:
                        updated_dict['upsert']['memory_data']['has_size'] = n_dict['tag']
                    else:
                        updated_dict['upsert']['memory_data']['has_tag'] = n_dict['tag']
            elif n_dict['action_type'] == 'ANSWER':
                updated_dict['dialogue_type'] = 'GET_MEMORY'
                if 'reference_object' not in n_dict or 'answer_type' not in n_dict:
                    skip_data.append([[text], re_dict])
                    continue
                updated_dict['filters'] = {'reference_object' : n_dict['reference_object'], 'type' : 'REFERENCE_OBJECT'}
                if n_dict['answer_type'] == 'QUERY_TAG':
                    if 'query_tag_name' in n_dict and 'query_tag_val' not in n_dict:
                        updated_dict['answer_type'] = 'TAG'
                        updated_dict['tag_name'] = 'has_' + n_dict['query_tag_name'].lower()
                    elif 'query_tag_name' in n_dict and 'query_tag_val' in n_dict:
                        updated_dict['answer_type'] = 'EXISTS'
                        val = n_dict['query_tag_val']
                        key = 'has_' + n_dict['query_tag_name'].lower()
                        updated_dict['filters']['reference_object'][key] = val
                    else:
                        skip_data.append([[text], re_dict])
                        continue
                elif n_dict['answer_type'] == 'QUERY_ALL_TAGS':
                    updated_dict['answer_type'] = 'TAG'
                    updated_dict['tag_name'] = 'has_name'
            elif n_dict['action_type'] == 'NOOP':
                updated_dict['dialogue_type'] = 'NOOP'
            else:
                updated_dict["dialogue_type"] = "HUMAN_GIVE_COMMAND"
                updated_dict["action"] = n_dict
            
            new_data+= [([text], updated_dict)]

    return new_data, skip_data

In [22]:
# for rephrases
rephrases = []
new_rephrases, skip_rephrases = [], []
rephrases = train_data['rephrased']
rephrases.extend(valid_data['rephrased'])
rephrases.extend(test_data['rephrased'])

# for x in rephrases:
#     if 'Name each very large item in front of the orange airport with big' in x[0]:
#         print(x)
new_rephrases, skip_rephrases = make_new_data(rephrases)
print(len(rephrases))
print(len(new_rephrases))
print(len(skip_rephrases))

35200
25402
9798


In [19]:
print(skip_rephrases[700])

(['what are the dimensions of the structure'], {'reference_object': {'location': {'location_type': 'SpeakerLook'}}, 'answer_type': 'query_tag', 'query_tag_name': 'size', 'action_type': 'Answer'})


In [13]:
# write things that need to be annotated

with open('/checkpoint/kavyasrinet/release_data_dialogue/rephrases.json', 'w') as f:
    json.dump(new_rephrases, f)

with open('/checkpoint/kavyasrinet/reannotate_data/rephrases.json', 'w') as f:
    json.dump(skip_rephrases, f)


In [15]:
prompts = []
prompts = valid_data['prompts']
prompts.extend(test_data['prompts'])

new_prompts, skip_prompts = make_new_data(prompts)
print(len(prompts))
print(len(new_prompts))
print(len(skip_prompts))

with open('/checkpoint/kavyasrinet/release_data_dialogue/prompts.json', 'w') as f:
    json.dump(new_prompts, f)

with open('/checkpoint/kavyasrinet/reannotate_data/prompts.json', 'w') as f:
    json.dump(skip_prompts, f)


3065
2513
552


In [16]:
# humanbot
humanbot = []
humanbot = valid_data['humanbot']
humanbot.extend(test_data['humanbot'])
new_humanbot, skip_humanbot = make_new_data(humanbot)
print(len(humanbot), len(new_humanbot), len(skip_humanbot))
with open('/checkpoint/kavyasrinet/release_data_dialogue/humanbot.json', 'w') as f:
    json.dump(new_humanbot, f)

with open('/checkpoint/kavyasrinet/reannotate_data/humanbot.json', 'w') as f:
    json.dump(skip_humanbot, f)


817 708 109


In [2]:
with open('dialogue_data.json') as f:
    dialogue_data = json.load(f)

In [6]:
templated = []
for key in ['train', 'valid', 'test']:
    data = dialogue_data[key]['templated']
    templated.extend(data)
print(len(templated))
print(templated[17])

with open('/checkpoint/kavyasrinet/release_data_dialogue/generated_dialogues.json')

800000
[['please reproduce 90 red porches over here'], {'dialogue_type': 'HUMAN_GIVE_COMMAND', 'action': {'reference_object': {'repeat': {'repeat_key': 'FOR', 'repeat_count': [0, [2, 2]]}, 'has_colour': [0, [3, 3]], 'has_name': [0, [4, 4]], 'has_attribute': [[0, [3, 3]], [0, [4, 4]]]}, 'location': {'coref_resolve': [0, [6, 6]]}, 'action_type': 'BUILD'}}]


In [124]:
# print(dialogue_data['test']['templated'][20])
print(len(dialogue_data['test']['templated']))
print(len(dialogue_data['train']['templated']))
for d_type in ['train', 'test', 'valid']:
    for example in dialogue_data[d_type]['templated']:
        if 'what are you doing' in example[0]:
            print(example)

5000
790000
[['what are you doing'], {'dialogue_type': 'GET_MEMORY', 'filters': {'temporal': 'CURRENT', 'type': 'ACTION'}, 'answer_type': 'TAG', 'tag_name': 'action_name'}]
[['what are you doing'], {'dialogue_type': 'GET_MEMORY', 'filters': {'temporal': 'CURRENT', 'type': 'ACTION'}, 'answer_type': 'TAG', 'tag_name': 'action_name'}]
[['what are you doing'], {'dialogue_type': 'GET_MEMORY', 'filters': {'temporal': 'CURRENT', 'type': 'ACTION'}, 'answer_type': 'TAG', 'tag_name': 'action_name'}]
[['what are you doing'], {'dialogue_type': 'GET_MEMORY', 'filters': {'temporal': 'CURRENT', 'type': 'ACTION'}, 'answer_type': 'TAG', 'tag_name': 'action_name'}]
[['what are you doing'], {'dialogue_type': 'GET_MEMORY', 'filters': {'temporal': 'CURRENT', 'type': 'ACTION'}, 'answer_type': 'TAG', 'tag_name': 'action_name'}]
[['what are you doing'], {'dialogue_type': 'GET_MEMORY', 'filters': {'temporal': 'CURRENT', 'type': 'ACTION'}, 'answer_type': 'TAG', 'tag_name': 'action_name'}]
[['what are you doing'

In [126]:
print(dialogue_data['train'].keys())

for x, y in zip(['rephrased'], [new_rephrases]): # add rephrases only
# for x, y in zip(['rephrased', 'prompts', 'humanbot'], [new_rephrases, new_prompts, new_humanbot]): 
    n_train = (9 * len(y)) // 10
    n_valid = len(y) // 20
    dialogue_data['train'][x] = y[ : n_train]
    dialogue_data['valid'][x] = y[n_train : n_train + n_valid]
    dialogue_data['test'][x] = y[n_train + n_valid : ]
    #print(x, len(dialogue_data['train'][x]), len(dialogue_data['valid'][x]), len(dialogue_data['test'][x]))
# write as train, valid, test and each is a dict -> rephrased, prompt, humanbot

dict_keys(['templated'])


In [127]:
print(dialogue_data['test']['rephrased'][29])

(['Complete the construction of a couple tiny stairways please'], {'dialogue_type': 'HUMAN_GIVE_COMMAND', 'action': {'reference_object': {'repeat': {'repeat_key': 'FOR', 'repeat_count': [0, [4, 5]]}, 'has_size': [0, [6, 6]], 'has_name': [0, [7, 7]]}, 'action_type': 'FREE_BUILD'}})


In [128]:
print(dialogue_data.keys())
# do also for templated
train_data, valid_data, test_data = dialogue_data['train'], dialogue_data['valid'], dialogue_data['test']
print(train_data.keys())
print(valid_data.keys())
print(test_data.keys())
total = {}
print(len(new_rephrases))
print(len(new_prompts))
print(len(new_humanbot))
for data_type in [train_data, valid_data, test_data]:
    for key in ['templated', 'rephrased']:
        
#     for key in ['templated', 'rephrased', 'prompts', 'humanbot']:
        print(key, len(data_type[key]))
    print("-----------------------------")
#         if key in data_type:
#             print(key, data_type[key])
#             total[key] = total.get(key, 0) + len(data_type[key])


dict_keys(['train', 'valid', 'test'])
dict_keys(['templated', 'rephrased'])
dict_keys(['templated', 'rephrased'])
dict_keys(['templated', 'rephrased'])
25402
2513
708
templated 790000
rephrased 22861
-----------------------------
templated 5000
rephrased 1270
-----------------------------
templated 5000
rephrased 1271
-----------------------------


In [129]:
with open('combined_dialogue_data.json', 'w') as f:
    json.dump(dialogue_data, f)

In [25]:
import json
with open('combined_dialogue_data.json') as f:
    data = json.load(f)

In [35]:
for data_type in ['train', 'valid', 'test']:
    for example in data[data_type]['rephrased']:
        if 'good job' in example[0]:
            print(example)

In [5]:
with open('/Users/kavyasrinet/rephrases.json') as f:
    data = json.load(f)

In [6]:
print(type(data))

<class 'list'>


In [7]:
def check_coref(d):
    for key, val in d.items():
        if key == "coref_resolve":
            return True
        if type(val) == dict:
            if check_coref(val):
                return True
        if type(val) == list and type(val[0]) == dict:
            for v in val:
                if check_coref(v):
                    return True
    return False

In [8]:
def location_type(d):
    
    for key, val in d.items():
        #print(key, val)
        if key == "location_type" and val in ['SPEAKER_LOOK', 'AGENT_POS', 'SPEAKER_POS']:
            return True
        if type(val) == dict:
            if location_type(val):
                return True
        if type(val) == list and type(val[0]) == dict:
            for v in val:
                if location_type(v):
                    return True
    return False

In [9]:
print(len(data))

35192


In [10]:
def update_coref(d):
    updated_d = {}
    for key, val in d.items():
        if key == "coref_resolve":
            updated_d['contains_coreference'] = "yes"
        elif type(val) == dict:
            updated_d[key] = update_coref(val)
        elif type(val) == list and type(val[0]) == dict:
            updated_d[key] = []
            for v in val:
                updated_d[key].append(update_coref(v))
        else:
            updated_d[key] = val
    return updated_d

In [11]:
def fix_freebuild(d):
    updated_d = {}
    for key, val in d.items():
        if key == "action_type" and val == 'FREE_BUILD':
            updated_d[key] = "FREEBUILD"
        elif type(val) == dict:
            updated_d[key] = fix_freebuild(val)
        elif type(val) == list and type(val[0]) == dict:
            updated_d[key] = []
            for v in val:
                updated_d[key].append(fix_freebuild(v))
        else:
            updated_d[key] = val
    return updated_d

In [12]:
def get_action_type(d):
    if 'action_sequence' in ad:
        return ad['action_sequence'][0]['action_type']
    return None

In [13]:
action_types = set()
for sentence, ad in data:
    if 'action_sequence' in ad:
        val = ad['action_sequence']
        if len(val)> 1:
            print("YESSSSS")
        for v in val:
#             if v['action_type'] == 'FREE_BUILD':
#                 print(sentence, ad)
            action_types.add(v['action_type'])
print(action_types)

{'DESTROY', 'STOP', 'UNDO', 'MOVE', 'DIG', 'FREE_BUILD', 'BUILD', 'FILL', 'RESUME'}


In [14]:
def fix_dig(ad):
    new_dict = {}
    action_dict = ad['action_sequence'][0]
    for key, val in action_dict.items():
        if key.startswith('has_'):
            if 'schematic' not in new_dict:
                new_dict['schematic'] = {}
            new_dict['schematic'][key] = val
        else:
            new_dict[key] = val
    out= {}
    out['dialogue_type'] = 'HUMAN_GIVE_COMMAND'
    out['action_sequence'] = [new_dict]
    return out

In [46]:
def check_target_action_type(d):
    for k, v in d.items():
        if k =='contains_coreference':
            return True
        if type(v) == dict:
            if check_target_action_type(v):
                return True
        if type(v) == list and type(v[0]) == dict:
            for val in v:
                if check_target_action_type(val):
                    return True
    return False

In [21]:
i = 0
j = 0
target = 0
updated_data = []
dig, fill, freebuild = 0, 0, 0
for sentence, ad in data:
    
    text = sentence
    new_d = ad
    
#     if check_target_action_type(ad):
#         print(sentence)
#         target += 1
    if get_action_type(ad) == 'DIG':
        dig += 1
        new_d = fix_dig(ad)
#     if get_action_type(ad) == 'FILL':
#         fill += 1
#         print(sentence, ad)
#         continue

    if get_action_type(new_d) == 'FREE_BUILD':
        freebuild += 1
        new_d = fix_freebuild(new_d)
        
#     if location_type(ad):
#         i +=1
    
    # contains coreference
    if check_coref(ad):
        new_d = update_coref(new_d)
    print(new_d)
    break
    updated_data+= [(text, new_d)]

print(i, dig, fill, freebuild)
print(target)

{'dialogue_type': 'HUMAN_GIVE_COMMAND', 'action_sequence': [{'location': {'reference_object': {'has_name': [0, [10, 10]]}, 'steps': [0, [2, 3]], 'relative_direction': 'LEFT', 'location_type': 'REFERENCE_OBJECT'}, 'action_type': 'MOVE'}]}
0 0 0 0
0


In [26]:
# get rephrases data
folder = '/Users/kavyasrinet/Desktop/annotated_data/'
with open(folder + 'train/rephrases.json') as f:
    rephrases = json.load(f)
with open(folder + 'valid/rephrases.json') as f:
    x = json.load(f)
    rephrases.extend(x)
with open(folder + 'test/rephrases.json') as f:
    x = json.load(f)
    rephrases.extend(x)
print(len(rephrases))

31870


In [29]:
def location_type(d):
    
    for key, val in d.items():
        #print(key, val)
        if key == "location_type" and val in ['SPEAKER_LOOK', 'AGENT_POS', 'SPEAKER_POS']:
            return True
        if type(val) == dict:
            if location_type(val):
                return True
        if type(val) == list and type(val[0]) == dict:
            for v in val:
                if location_type(v):
                    return True
    return False

In [65]:
speaker_look = []
others = []
cnt  = 0
for r in rephrases:
    sent, ad = r
    if location_type(ad):
        speaker_look.append(r)
    else:
        others.append(r)
print(cnt)

0


In [66]:
print(len(speaker_look))
print(len(others))

2359
29511


In [37]:
with open('/Users/kavyasrinet/Desktop/speaker_look.txt', 'w') as f:
    for x in speaker_look:
        sent, tree = x
        f.write(sent+"\n")
        f.write(json.dumps(tree) + "\n")
        f.write("\n")

In [67]:
new_r = []
with open('/Users/kavyasrinet/Desktop/speaker_look.txt') as f:
    i = 0
    text = None
    ad = None
    for line in f.readlines():
        line = line.strip()
        i += 1
        if len(line) == 0:
            new_r.append([text, ad])
            text = None
            ad = None
        elif line.startswith('{'):
            ad = json.loads(line)
        else:
            text = line
            
print(len(new_r))

2359


In [70]:
others.extend(new_r)
print(len(others))

31870


In [96]:
def find_keys(d):
    for key, val in d.items():
        if key.endswith('_'):
            return True
        if type(val) == dict:
            if find_keys(val):
                return True
        elif type(val) == list and type(val[0]) ==dict:
            for v in val:
                if find_keys(v):
                    return True
        
    return False

In [97]:
def fix_keys(d):
    ad = {}
    for key, val in d.items():
        if key.endswith('_'):
            ad[key[:-1]] = val
        elif type(val) == dict:
            ad[key]  = fix_keys(val)
        elif type(val) == list and type(val[0]) ==dict:
            ad[key] = []
            for v in val:
                ad[key].append(fix_keys(v))
        else:
            ad[key] = val
    return ad

In [101]:
updated_others = []
i =0
for item in others:
    sent, ad = item
    if find_keys(ad):
        new_ad = fix_keys(ad)
        updated_others.append([sent, new_ad])
    else:
        updated_others.append([sent, ad])

In [102]:
print(len(updated_others))
for item in updated_others:
    sent, ad = item
    if find_keys(ad):
        print(ad)

31870


In [103]:
with open('/Users/kavyasrinet/Github/minecraft/python/craftassist/ttad/rephrases_data/rephrases.json', 'w') as f:
    json.dump(updated_others, f)

In [104]:
with open('/Users/kavyasrinet/Github/minecraft/python/craftassist/ttad/rephrases_data/rephrases.json') as f:
    data = json.load(f)

In [108]:
print(type(data[30][1]))

<class 'dict'>
